## Архитектуры свёрточных сетей

In [5]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary
import time
import matplotlib.pyplot as plt

def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        
        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 10 == 0:
                print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. " 
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Define your datasets and data loaders here
BATCH_SIZE = 192

# Define transformations for your datasets here
transoforms_emnist = tv.transforms.Compose([
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

# Example: EMNIST balanced dataset
train_dataset_emnist = tv.datasets.EMNIST('../data', split='balanced', train=True, download=True, transform=transoforms_emnist)
test_dataset_emnist = tv.datasets.EMNIST('../data', split='balanced', train=False, download=True, transform=transoforms_emnist)

train_iter_emnist = torch.utils.data.DataLoader(train_dataset_emnist, batch_size=BATCH_SIZE)
test_iter_emnist = torch.utils.data.DataLoader(test_dataset_emnist, batch_size=BATCH_SIZE)

# Define your model architectures here

# Example: ResNet18
model_resnet18 = tv.models.resnet18(pretrained=True)
for param in model_resnet18.parameters():
    param.requires_grad = False
model_resnet18.fc = nn.Linear(in_features=512, out_features=10)
trainer_resnet18 = torch.optim.Adam(model_resnet18.fc.parameters(), lr=0.001)

# Example: VGG16
model_vgg16 = tv.models.vgg16(pretrained=True)
for param in model_vgg16.parameters():
    param.requires_grad = False
model_vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 10)
)
trainer_vgg16 = torch.optim.Adam(model_vgg16.classifier.parameters(), lr=0.001)

# Example: DenseNet161
model_densenet161 = tv.models.densenet161(pretrained=True)
for param in model_densenet161.parameters():
    param.requires_grad = False
model_densenet161.classifier = nn.Linear(in_features=2208, out_features=10)
trainer_densenet161 = torch.optim.Adam(model_densenet161.classifier.parameters(), lr=0.001)

# Example: InceptionV3
BATCH_SIZE = 192
transforms_inception = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((299, 299)),
    tv.transforms.ToTensor()
])

train_dataset_inception = tv.datasets.EMNIST('.', train=True, transform=transforms_inception, download=True, split='mnist')
test_dataset_inception = tv.datasets.EMNIST('.', train=False, transform=transforms_inception, download=True, split='mnist')

train_iter_inception = torch.utils.data.DataLoader(train_dataset_inception, batch_size=BATCH_SIZE)
test_iter_inception = torch.utils.data.DataLoader(test_dataset_inception, batch_size=BATCH_SIZE)

model_inception_v3 = tv.models.inception_v3(pretrained=True)
for param in model_inception_v3.parameters():
    param.requires_grad = False
model_inception_v3.fc = nn.Linear(in_features=2048, out_features=10)
trainer_inception_v3 = torch.optim.Adam(model_inception_v3.fc.parameters(), lr=0.001)

# Compare the models and plot training loss
def compare_models(models, train_iters, test_iters, trainers, num_epochs, model_names):
    train_losses = []
    test_accuracies = []

    for i, model in enumerate(models):
        print(f"Training {model_names[i]}...")
        train_loss = train(model, train_iters[i], test_iters[i], trainers[i], num_epochs)
        train_losses.append(train_loss)

        test_acc = evaluate_accuracy(test_iters[i], model.to(device))
        test_accuracies.append(test_acc)

    # Plot training loss for each model
    plt.figure(figsize=(10, 5))
    for i, loss in enumerate(train_losses):
        plt.plot(loss, label=model_names[i])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.title('Training Loss Comparison')
    plt.show()

    # Create a table to compare test accuracies
    print("| Model | Test Accuracy |")
    print("| - | - |")
    for i in range(len(model_names)):
        print(f"| {model_names[i]} | {test_accuracies[i]:.3f} |")

# Define your models, train iterators, test iterators, trainers, and model names here
models = [model_resnet18, model_vgg16, model_densenet161, model_inception_v3]
train_iters = [train_iter_emnist, train_iter_emnist, train_iter_emnist, train_iter_inception]
test_iters = [test_iter_emnist, test_iter_emnist, test_iter_emnist, test_iter_inception]
trainers = [trainer_resnet18, trainer_vgg16, trainer_densenet161, trainer_inception_v3]
model_names = ['ResNet18', 'VGG16', 'DenseNet161', 'InceptionV3']

compare_models(models, train_iters, test_iters, trainers, num_epochs=10, model_names=model_names)


c:\Users\Анатолий\source\repos\PyTorchtest\PyTorchtest\TorchEnv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\Анатолий\source\repos\PyTorchtest\PyTorchtest\TorchEnv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\Анатолий\source\repos\PyTorchtest\PyTorchtest\TorchEnv\Lib\site-packages\torchvision\models\_utils.py

Training ResNet18...


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
